In [1]:
!pip install wavio
!pip install sounddevice
!pip install torch accelerate torchaudio datasets
!pip install --upgrade transformers
!pip install -q torchaudio omegaconf
!pip install flask-ngrok
!pip install pyngrok
!ngrok config add-authtoken 2U1f33A1OBVeXaNZt9cSlJanE4f_7W4e2HoUeQ2iqrwZGpy2Y

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [2]:
!pip install -U openai-whisper

In [3]:
!pip install gtts

In [4]:
import pandas as pd


In [5]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import re

def preprocess(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    text = re.sub(r'[^\w\s]', '', text)  # Remove non-alphanumeric characters
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    return ' '.join(stemmed_tokens)

In [7]:
def preprocess_with_stopwords(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    text = re.sub(r'[^\w\s]', '', text)  # Remove non-alphanumeric characters
    tokens = nltk.word_tokenize(text.lower())
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    return ' '.join(stemmed_tokens)

In [8]:
import pandas as pd
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import ISRIStemmer
import re


In [9]:
#...............IMPORTING ASR MODEL.............
import whisper
model = whisper.load_model("tiny")

In [10]:
from transformers import (Wav2Vec2Processor, Wav2Vec2ForCTC)
import torchaudio
import torch

def speech_file_to_array_fn(voice_path, resampling_to=16000):
    speech_array, sampling_rate = torchaudio.load(voice_path)
    resampler = torchaudio.transforms.Resample(sampling_rate, resampling_to)

    return resampler(speech_array)[0].numpy(), sampling_rate

# load the model
cp = "bakrianoo/sinai-voice-ar-stt"
processor = Wav2Vec2Processor.from_pretrained(cp)
modelA = Wav2Vec2ForCTC.from_pretrained(cp)

In [11]:
import sys
sys.executable

'/usr/bin/python3'

In [12]:
#..............IMPORTING TTS MODEL..............

from gtts import gTTS
from IPython.display import Audio

In [13]:
import time
import os
import torch
print(os.getcwd())
def calculate_execution_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Function '{func.__name__}' execution time: {execution_time:.6f} seconds")
        return result
    return wrapper


/content


In [14]:
import scipy.io.wavfile as wav
from scipy.io import wavfile
import numpy as np

In [19]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify, render_template, send_file, flash, request, redirect, url_for
import os
import soundfile as sf
import io
import base64
import scipy.signal as signal
import torch
import torchaudio

app = Flask(__name__, template_folder='templateFiles', static_folder='staticFiles')
run_with_ngrok(app)

UPLOAD_FOLDER = "/content/ALAM"
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif', "csv"}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# ML model logic
def ml_model_prediction(audio_blob, questions_list, answers_list):

    audio_blob.seek(0)


    audio_data, _ = sf.read(io.BytesIO(audio_blob.read()))
    sf.write("outputx.wav", audio_data,48000 )

    # Language Identification
    audio = whisper.load_audio("outputx.wav")
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)
    language = max(probs, key=probs.get)
    print(f"Detected language:", language)

    if language == "fr" or language == "en" or language == "de":
      result = model.transcribe("outputx.wav")
      transcription = result["text"]
      print("Transcription", transcription)
    else:
      sample, sr = speech_file_to_array_fn("outputx.wav")
      inputs = processor([sample], sampling_rate=16_000, return_tensors="pt", padding=True)

      with torch.no_grad():
          logits = modelA(inputs.input_values,).logits

      predicted_ids = torch.argmax(logits, dim=-1)
      text = processor.batch_decode(predicted_ids)
      transcription = ' '.join(text)

      print("Transcription:",transcription)



    #...................................................................

    vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)
    X = vectorizer.fit_transform([preprocess(q) for q in questions_list])

    def get_response(text):
      processed_text = preprocess_with_stopwords(text)
      #print("processed_text:", processed_text)
      vectorized_text = vectorizer.transform([processed_text])
      similarities = cosine_similarity(vectorized_text, X)
      #print("similarities:", similarities)
      max_similarity = np.max(similarities)
      #print("max_similarity:", max_similarity)
      if max_similarity > 0.6:
          high_similarity_questions = [q for q, s in zip(questions_list, similarities[0]) if s > 0.6]
          #print("high_similarity_questions:", high_similarity_questions)

          target_answers = []
          for q in high_similarity_questions:
              q_index = questions_list.index(q)
              target_answers.append(answers_list[q_index])
          #print(target_answers)

          Z = vectorizer.fit_transform([preprocess_with_stopwords(q) for q in high_similarity_questions])
          processed_text_with_stopwords = preprocess_with_stopwords(text)
          #print("processed_text_with_stopwords:", processed_text_with_stopwords)
          vectorized_text_with_stopwords = vectorizer.transform([processed_text_with_stopwords])
          final_similarities = cosine_similarity(vectorized_text_with_stopwords, Z)
          closest = np.argmax(final_similarities)
          return target_answers[closest]
      else:
          return "I can't answer this question."


    f_text = get_response(transcription)
    print("BOT_ANSWER", f_text)



    #...................................................................

    if language == "fr":
      audio = gTTS(f_text , lang = "fr")
    elif language == "en":
      audio = gTTS(f_text , lang = "en")
    elif language == "de":
      audio = gTTS(f_text , lang = "de")
    else:
      audio = gTTS(f_text , lang = "ar")



    #...................................................................

    audio.save('output.wav') #save the string converted to speech as a .wav file
    sound_file = 'output.wav'
    Audio(sound_file, autoplay=True)
    audio_path = '/content/output.wav'
    return send_file(audio_path, mimetype='output/wav')


# Serve the index.html file

@app.route('/')
@app.route('/main')
def main():
    return render_template('bravo.html')

@app.route('/charlie1')
def charlie1():
    return render_template('lala -21.html')

@app.route('/charlie2')
def charlie2():
    return render_template('lala -22.html')
@app.route('/charlie3')
def charlie3():
    return render_template('lala -23.html')
@app.route('/charlie4')
def charlie4():
    return render_template('lala -24.html')


@app.route('/success', methods = ['POST'])
def success():
    if request.method == 'POST':
        f = request.files['file']
        f.save(f.filename)
        return render_template('bravo.html')

# @app.route('/index')
# def index():
#     return render_template('lala -2.html')

# API endpoint for audio file upload and ML model prediction
@app.route('/predict', methods=['POST'])
def predict():

    df = pd.read_csv("TEXT.csv", encoding='utf-8')
    questions_list = df['Questions'].tolist()
    answers_list = df['Answers'].tolist()
    print("Audio Recieved!")

    audio_blob = request.files['audio']

    if not audio_blob:
        return jsonify({'error': 'No audio file provided.'}), 400

    # Perform prediction using ML model
    predictions = ml_model_prediction(audio_blob, questions_list, answers_list)
    return predictions


if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://01eb-34-91-4-67.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [27/Aug/2023 20:46:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Aug/2023 20:46:23] "GET /staticFiles/bravo.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Aug/2023 20:46:38] "POST /success HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Aug/2023 20:46:38] "GET /staticFiles/bravo.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [27/Aug/2023 20:46:42] "GET /charlie2 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Aug/2023 20:46:43] "GET /staticFiles/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Aug/2023 20:46:43] "GET /staticFiles/index2.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Aug/2023 20:46:43] "GET /staticFiles/index2.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Aug/2023 21:20:21] "GET /charlie2 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Aug/2023 21:20:22] "GET /staticFiles/index2.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Aug/2023 21:20:22] "GET /staticFiles/index2.js HTTP/1.1" 200 -
INFO:werkzeug